In [1]:
import numpy as np
from matplotlib import pyplot as plt
from abmn_graph_classes import *

### Equations:

\begin{equation} \tag{1}
m_i^{(t)} = \frac{a_i^{(t)}}{a_i^{(t)}+b_i^{(t)}} m_{i+1}^{(t-1)} + \frac{b_i^{(t)}}{a_i^{(t)}+b_i^{(t)}} m_{i-1}^{(t-1)} - a_i^{(t)}
\end{equation}

\begin{equation*} \tag{2}
n_i^{(t)} = \frac{a_i^{(t)}}{a_i^{(t)}+b_i^{(t)}} n_{i+1}^{(t-1)} + \frac{b_i^{(t)}}{a_i^{(t)}+b_i^{(t)}} n_{i-1}^{(t-1)} - b_i^{(t)}
\end{equation*}

\begin{equation*} \tag{3}
(a_i^{(t)} + b_i^{(t)})^2 = b_i^{(t)} (m_{i+1}^{(t-1)} - m_{i-1}^{(t-1)})
\end{equation*}

\begin{equation*} \tag{4}
(a_i^{(t)} + b_i^{(t)})^2 = a_i^{(t)} (n_{i-1}^{(t-1)} - n_{i+1}^{(t-1)})
\end{equation*}

### Solving:

Strategy: at each time $t$, suppose we have $m_{i+1}^{(t-1)}$, $m_{i-1}^{(t-1)}$ and $n_{i+1}^{(t-1)}$, $n_{i-1}^{(t-1)}$. <br>
Then, first solve for $a_i^{(t)}$ and $b_i^{(t)}$ using eq (3) and eq (4). <br>
Plug into to eq (1) and eq (2) to solve for $m_{i}^{(t)}$ and $n_{i}^{(t)}$.

Equating (3) and (4):

\begin{equation*}
a_i^{(t)} (n_{i-1}^{(t-1)} - n_{i+1}^{(t-1)}) = b_i^{(t)} (m_{i+1}^{(t-1)} - m_{i-1}^{(t-1)})
\end{equation*}

\begin{equation*}
a_i^{(t)} = \frac{(m_{i+1}^{(t-1)} - m_{i-1}^{(t-1)})}{(n_{i-1}^{(t-1)} - n_{i+1}^{(t-1)})} b_i^{(t)}
\end{equation*}

\begin{equation*}
a_i^{(t)} = \frac{\Delta m}{\Delta n} b_i^{(t)}
\end{equation*}

where $\Delta m$ and $\Delta n$ are calculable constants obtained from previous timestep: 

\begin{equation*}
\Delta m = (m_{i+1}^{(t-1)} - m_{i-1}^{(t-1)})
\end{equation*}

\begin{equation*}
\Delta n = (n_{i-1}^{(t-1)} - n_{i+1}^{(t-1)})
\end{equation*}

Expanding (3):

\begin{equation*}
(a_i^{(t)})^2 + 2 a_i^{(t)} b_i^{(t)} + (b_i^{(t)})^2 = b_i^{(t)} (m_{i+1}^{(t-1)} - m_{i-1}^{(t-1)})
\end{equation*}

Substituting expression for $a_i^{(t)}$ in terms of $b_i^{(t)}$:

\begin{equation*}
(\frac{\Delta m}{\Delta n} b_i^{(t)})^2 + 2 (\frac{\Delta m}{\Delta n} b_i^{(t)}) b_i^{(t)} + (b_i^{(t)})^2 = b_i^{(t)} \Delta m
\end{equation*}

\begin{equation*}
(b_i^{(t)})^2 \left( \left(\frac{\Delta m}{\Delta n} \right)^2 + 2\frac{\Delta m}{\Delta n} + 1 \right) = b_i^{(t)} \Delta m
\end{equation*}

if $b_i^{(t)} \neq 0$:

\begin{equation*}
b_i^{(t)} = \frac{\Delta m}{\left( \left(\frac{\Delta m}{\Delta n} \right)^2 + 2\frac{\Delta m}{\Delta n} + 1 \right)}
\end{equation*}

\begin{equation*}
= \frac{\Delta m}{\left( \frac{\Delta m}{\Delta n} + 1 \right)^2}
\end{equation*}

which means:

\begin{equation*}
a_i^{(t)} = \frac{\Delta m}{\Delta n} \frac{\Delta m}{\left( \frac{\Delta m}{\Delta n} + 1 \right)^2}
\end{equation*}

Finally, we can plugin values of $a_i^{(t)}$ and $b_i^{(t)}$ into eq (1) and eq (2) to obtain $m_i^{(t)}$ and $n_i^{(t)}$.

### Initialization:

#### Contants:
Left end = $-k$ <br>
Right end = $l$ <br>
Payoff factor = $\lambda$ <br>

#### Positional Boundary Values:
$n_{-k}^{(t)} = 1$ -------- $n_{l}^{(t)} = 0$ <br>
$m_{-k}^{(t)} = 0$ -------- $m_{l}^{(t)} = \lambda$ <br>
for all time step $t$ <br>

#### Time Boundary Values:
$m_i^{(0)} = \frac{\lambda}{k+l}(i+k)$ <br>
$n_i^{(0)} = \frac{l-i}{k+l}$ <br>
for all positions $-k \leq i \leq l$ <br>

In [115]:
def initializeLine(right = 6, left = 6, time = 100, maxi_reward = 100000, mina_reward = 99994):
    k = right     # left endpoint: -k
    l = left      # right endpoint: l
    L = k+l+1    # total length: k+l
    T = time   # Time steps

    # Note: position index i will be shifted
    # before: left end = -k, middle = 0, right end = l
    # now: left end = 0, middle = k, right end = k + l (= L)

    m_lambda = maxi_reward
    n_lambda = mina_reward


    # initialize the m,n values at time t=0, 
    # then iterate starting from t=1, use m(t-1), n(t-1) to compute a(t) and b(t) 
    # then use a(t) and b(t) to compute m(t) and n(t)

    # Each matrix: vertical axis (rows): time step, horizontal axis (colunmns): position
    # Therefore, m[t][i] access m value at time t, position index i
    a = np.zeros(T*L).reshape((T, L))
    b = np.zeros(T*L).reshape((T, L))
    m = np.zeros(T*L).reshape((T, L))
    n = np.zeros(T*L).reshape((T, L))

    # Initialize positional boundary values
    for t in range(a.shape[0]):
        n[t][0] = n_lambda
        m[t][0] = 0
        n[t][L-1] = 0
        m[t][L-1] = m_lambda
        
    # Initialize time boundary values
    for i in range(L):
        t = i / (L - 1)
        
        if i == 0:
            m[0][i] = 0
            n[0][i] = n_lambda
        elif i == L - 1:
            m[0][i] = m_lambda
            n[0][i] = 0
        else:

            m[0][i] = m_lambda * (np.arctan(10 * t - 5) / np.pi + 0.5)
            n[0][i] = n_lambda * (np.arctan(5-(10 * t)) / np.pi + 0.5)
    return a,b,m,n
a,b,m,n = initializeLine()

In [ ]:
print("a[0]:", a[0])
print("b[0]:", b[0])
print("m[0]:", m[0])
print("n[0]:", n[0])
print("m[1]:", m[1])
print("n[1]:", n[1])

In [117]:
# Recursive solving algorithm
def iterateAMBN(a, b, m, n, t, DEBUG = False):
    L = a.shape[1]
    if DEBUG:
        print(f"-------------- step {t} --------------")
    for i in range(1, L-1):
        delta_m = m[t-1][i+1] - m[t-1][i-1]
        delta_n = n[t-1][i-1] - n[t-1][i+1]
        # Compute a,b using previous m,n
        b[t][i] = delta_m / (((delta_m / delta_n) + 1))
        b[t][i] /= ((delta_m / delta_n) + 1)
        a[t][i] = (delta_m/delta_n) * b[t][i]
        # Compute current m,n using current a,b
        m[t][i] = (a[t][i]/(a[t][i] + b[t][i]))*m[t-1][i+1] + (b[t][i]/(a[t][i] + b[t][i]))*m[t-1][i-1] - a[t][i]
        n[t][i] = (a[t][i]/(a[t][i] + b[t][i]))*n[t-1][i+1] + (b[t][i]/(a[t][i] + b[t][i]))*n[t-1][i-1] - b[t][i]
    
        if DEBUG:
            print("Previous m, n:")
            print(m[t-1][i+1], m[t-1][i-1], n[t-1][i+1], n[t-1][i-1])
            print("Delta m, Delta n:")
            print(delta_m, delta_n)
            print("Current a, b:")
            print(a[t][i], b[t][i])
            print("Current m, n:")
            print(m[t][i], n[t][i])
#iterateAMBN(a,b,m,n)

def propogateAMBN(a,b,m,n,T,Debug = False):
    for t in range(1,T):
        iterateAMBN(a,b,m,n,t,Debug)
propogateAMBN(a,b,m,n,100)


In [ ]:
b[20][4]

# Junction:

At the junction point, we must first pick a direction.

\begin{align}
v^{+} &= \arg \max_{*}(m^{(t-1)}_{*}) \tag{5} \\
v^{-} &= \arg \max_{*}(n^{(t-1)}_{*}) \notag \\
* &= \text{neighbors}(v) \notag
\end{align}

Right now, we work on the case where the graph is a single junction. This means the graph i Z from -6 to 6 with 6 more nodes coming out out the top and bottom at index 0

In [87]:
#note, this code only works because the junction is the same index in both directions
def pickDirection(junc, m1, m2, time):
    values = []

    if junc - 1 >= 0 and junc - 1 < len(m1[0]) and time < len(m1):
        values.append(("left", m1[time-1][junc - 1]))
    if junc + 1 >= 0 and junc + 1 < len(m1[0]) and time < len(m1):
        values.append(("right", m1[time-1][junc + 1]))
    if junc - 1 >= 0 and junc - 1 < len(m2[0]) and time < len(m2):
        values.append(("down", m2[time-1][junc - 1]))
    if junc + 1 >= 0 and junc + 1 < len(m2[0]) and time < len(m2):
        values.append(("up", m2[time-1][junc + 1]))

    if values:
        direction, _ = max(values, key=lambda x: x[1])
        return direction
    else:
        return None


\begin{equation} \tag{1}
m_v^{(t)} = \frac{a_v^{(t)}}{a_v^{(t)}+b_v^{(t)}} m_{v+}^{(t-1)} + \frac{b_v^{(t)}}{a_v^{(t)}+b_v^{(t)}} m_{v-}^{(t-1)} - a_v^{(t)}
\end{equation}

\begin{equation*} \tag{2}
n_v^{(t)} = \frac{a_v^{(t)}}{a_v^{(t)}+b_v^{(t)}} n_{v+}^{(t-1)} + \frac{b_v^{(t)}}{a_v^{(t)}+b_v^{(t)}} n_{v-}^{(t-1)} - b_v^{(t)}
\end{equation*}

\begin{equation*} \tag{3}
(a_v^{(t)} + b_v^{(t)})^2 = b_v^{(t)} (m_{v+}^{(t-1)} - m_{v-}^{(t-1)})
\end{equation*}

\begin{equation*} \tag{4}
(a_v^{(t)} + b_v^{(t)})^2 = a_v^{(t)} (n_{v-}^{(t-1)} - n_{v+}^{(t-1)})
\end{equation*}

\begin{equation*}
\Delta m = (m_{v+}^{(t-1)} - m_{v-}^{(t-1)})
\end{equation*}

\begin{equation*}
\Delta n = (n_{v-}^{(t-1)} - n_{v+}^{(t-1)})
\end{equation*}

We solved this essentially before, im just going to slightly shift the code

In [119]:
a1,b1,m1,n1 = initializeLine() #leftright
a2,b2,m2,n2 = initializeLine() #updown
def iterateJunction(a, b, m1, n1, m2, n2, t, DEBUG = False):
    L = a.shape[1]
    junc = L//2
    v_plus = pickDirection(junc, m1, m2, t)
    v_minus = pickDirection(junc, n1, n2, t)
    if v_plus is None or v_minus is None:
        return
    m_plus, n_plus, m_minus, n_minus = 0, 0, 0, 0
    if v_plus == "up":
        m_plus = m2[t-1][junc+1]
        n_plus = n2[t-1][junc+1]
    elif v_plus == "down":
        m_plus = m2[t-1][junc-1]
        n_plus = n2[t-1][junc-1]
    elif v_plus == "right":
        m_plus = m1[t-1][junc+1]
        n_plus = n1[t-1][junc+1]
    elif v_plus == "left":
        m_plus = m1[t-1][junc-1]
        n_plus = n1[t-1][junc-1]
    
    if v_minus == "up":
        m_minus = m2[t-1][junc+1]
        n_minus = n2[t-1][junc+1]
    elif v_minus == "down":
        m_minus = m2[t-1][junc-1]
        n_minus = n2[t-1][junc-1]
    elif v_minus == "right":
        m_minus = m1[t-1][junc+1]
        n_minus = n1[t-1][junc+1]
    elif v_minus == "left":
        m_minus = m1[t-1][junc-1]
        n_minus = n1[t-1][junc-1]
    if DEBUG:
        print(f"-------------- step {t} --------------")
    for i in range(1, L-1):
        if i == junc:
            delta_m = m_plus - m_minus
            delta_n = n_minus - n_plus
        else:
            delta_m = m1[t-1][i+1] - m1[t-1][i-1]
            delta_n = n1[t-1][i-1] - n1[t-1][i+1]
        # Compute a,b using previous m,n
        b[t][i] = delta_m / (((delta_m / delta_n) + 1))
        b[t][i] /= ((delta_m / delta_n) + 1)
        a[t][i] = (delta_m/delta_n) * b[t][i]
        # Compute current m,n using current a,b
        if i == junc:
            m1[t][i] = (a[t][i]/(a[t][i] + b[t][i]))*m_plus + (b[t][i]/(a[t][i] + b[t][i]))*m_minus - a[t][i]
            n1[t][i] = (a[t][i]/(a[t][i] + b[t][i]))*n_plus + (b[t][i]/(a[t][i] + b[t][i]))*n_minus - b[t][i]
        else:
            m1[t][i] = (a[t][i]/(a[t][i] + b[t][i]))*m1[t-1][i+1] + (b[t][i]/(a[t][i] + b[t][i]))*m1[t-1][i-1] - a[t][i]
            n1[t][i] = (a[t][i]/(a[t][i] + b[t][i]))*n1[t-1][i+1] + (b[t][i]/(a[t][i] + b[t][i]))*n1[t-1][i-1] - b[t][i]
    
    
        if DEBUG:
            print("Previous m, n:")
            print(m1[t-1][i+1], m1[t-1][i-1], n1[t-1][i+1], n1[t-1][i-1])
            print("Delta m, Delta n:")
            print(delta_m, delta_n)
            print("Current a, b:")
            print(a1[t][i], b1[t][i])
            print("Current m, n:")
            print(m1[t][i], n1[t][i])

def propogateJunction(a1,b1,a2,b2,m1,n1,m2,n2,T):
    for t in range(1,T):
        iterateJunction(a1,b1,m1,n1,m2,n2,t)
        iterateJunction(a2,b2,m2,n2,m1,n1,t) # possible issue is direction change here
propogateJunction(a1,b1,a2,b2,m1,n1,m2,n2,100)


In [ ]:
b2[20][4]


In [108]:
def find_batteground(a, b, t, tolerance_ratio = 10):
    battleground = []

    for i in range(len(a[0])):
        m = max(a[t][i] , b[t][i])
        n = min(a[t][i] , b[t][i])
        ratio = m / n if n != 0 else float('inf')
        if ratio < tolerance_ratio:
            print(ratio)
            battleground.append(i)

    return battleground

In [109]:
a,b,m,n = initializeLine(maxi_reward=100000, mina_reward=100000)
propogateAMBN(a,b,m,n,100)

In [ ]:
print(a[50][1], b[50][1])
a[50][1]/b[50][1]

In [ ]:
for t in range(50,51):
    print(find_batteground(a,b,t))

In [ ]:
for t, a_line in enumerate(a):
    plt.plot(list(range(-k,l+1)), a_line, label=t)

plt.legend()
plt.show()

In [ ]:
plt.imshow(a)

In [ ]:
plt.imshow(m)

In [5]:
from abmn_graph_classes import *
cross = OriginCrossGraph(left=6,
                         right=6,
                         top=6,
                         bottom=6,
                         time_steps=4000,
                         horizontal_maxi_reward=1,
                         horizontal_mina_reward=0.99994,
                         vertical_maxi_reward=1,
                         vertical_mina_reward=1)